# Deploy LLaVA on Amazon Sagemaker

In [2]:
!pip install sagemaker --upgrade --quiet

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


### Deploy Llava

In [5]:
import sagemaker
from sagemaker.huggingface.model import HuggingFaceModel

LLAVA_MODEL = "s3://public-assets-vincent-claes/llava-v1.5-13b/model.tar.gz"

huggingface_model = HuggingFaceModel(
   model_data=LLAVA_MODEL,      # path to your model and script
   role=sagemaker.get_execution_role(), # iam role with permissions to create an Endpoint
   transformers_version="4.28.1",  # transformers version used
   pytorch_version="2.0.0",       # pytorch version used
   py_version='py310',            # python version used
   model_server_workers=1
)

predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type="ml.g5.12xlarge",
    container_startup_health_check_timeout=900, # increase timeout for large models
    model_data_download_timeout=900, # increase timeout for large models
)

----------------!

In [8]:
### Call Llava

In [9]:
import base64
import requests

# URL of llava logo image
url = "https://raw.githubusercontent.com/haotian-liu/LLaVA/main/images/llava_logo.png"

# Fetching the image from the URL
response = requests.get(url)
image_data = response.content
encoded_image = base64.b64encode(image_data).decode()

# request
data = {
    "image" : encoded_image,
    "question" : "Describe the image and color details.",
}
output = predictor.predict(data)
print(output)

The image features a small, red toy animal, possibly a stuffed animal or a figurine, with flames coming out of its back. The animal is wearing glasses, giving it a unique and eye-catching appearance. The flames are orange and yellow, adding a sense of warmth and energy to the scene. The overall color palette of the image is predominantly red, with the toy and the flames being the main sources of this color.
